In [765]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import re

In [766]:
mass = pd.read_csv('../source/mmALL_073120_csv.csv')
mass.head(3)

,id,country,ccode,year,region,protest,protestnumber,startday,startmonth,startyear,...,protesterdemand4,stateresponse1,stateresponse2,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes
0,201990001,Canada,20,1990,North America,1,1,15.0,1.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...
1,201990002,Canada,20,1990,North America,1,2,25.0,6.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...
2,201990003,Canada,20,1990,North America,1,3,1.0,7.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. quebec protest after queen calls for unity ...,"the queen, after calling on canadians to remai..."


In [767]:
mass[mass['country']=='Czech Republic'][['country','startmonth','startyear','endmonth','endyear','protesterdemand1','stateresponse1']].head()

,country,startmonth,startyear,endmonth,endyear,protesterdemand1,stateresponse1
5362,Czech Republic,NaN,NaN,NaN,NaN,NaN,NaN
5363,Czech Republic,3.0,1994.0,3.0,1994.0,"political behavior, process",ignore
5364,Czech Republic,5.0,1995.0,5.0,1995.0,police brutality,crowd dispersal
5365,Czech Republic,11.0,1995.0,11.0,1995.0,labor wage dispute,ignore
5366,Czech Republic,12.0,1996.0,12.0,1996.0,social restrictions,ignore


In [768]:
mass = mass[mass['protest']==1]

In [769]:
mass.columns

Index(['id', 'country', 'ccode', 'year', 'region', 'protest', 'protestnumber',
       'startday', 'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
       'protesterviolence', 'location', 'participants_category',
       'participants', 'protesteridentity', 'protesterdemand1',
       'protesterdemand2', 'protesterdemand3', 'protesterdemand4',
       'stateresponse1', 'stateresponse2', 'stateresponse3', 'stateresponse4',
       'stateresponse5', 'stateresponse6', 'stateresponse7', 'sources',
       'notes'],
      dtype='object')

In [770]:
mass.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15239 entries, 0 to 17141
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     15239 non-null  int64  
 1   country                15239 non-null  object 
 2   ccode                  15239 non-null  int64  
 3   year                   15239 non-null  int64  
 4   region                 15239 non-null  object 
 5   protest                15239 non-null  int64  
 6   protestnumber          15239 non-null  int64  
 7   startday               15239 non-null  float64
 8   startmonth             15239 non-null  float64
 9   startyear              15239 non-null  float64
 10  endday                 15239 non-null  float64
 11  endmonth               15239 non-null  float64
 12  endyear                15239 non-null  float64
 13  protesterviolence      15239 non-null  float64
 14  location               15218 non-null  object 
 15  pa

In [771]:
mass.participants_category.value_counts()

100-999       3204
50-99         2508
2000-4999     1580
>10000        1470
5000-10000     642
1000-1999      483
Name: participants_category, dtype: int64

In [772]:
mass.participants_category.isnull().sum()

5352

In [773]:
mass['participants'].isnull().sum()

12

In [774]:
def string_to_number(string):

    # if nan then just exit
    if string is np.nan:
        return string
    
    output = re.findall (  r'\d+(?:,\d+)?' , string)
        
    if len(output ) == 0:
        return string
    elif len(output) == 1:
        return output[0]
    elif len(output) > 1:
        return max(output)
    else:
        return output

In [775]:
#re.findall (  r'\d+(?:,\d+)?' , '50,00')

In [776]:

#Used to convert participants column from a mess of text into nice clean numbers
# Return value is still a string value, but can be converted easily later

def participants_to_numbers(string):

    # if nan then just exit
    if string is np.nan:
        return string
    
    if type(string) is list:
        return string
    
    string = string.lower()
    
    small_groups = ['dozen','dozens', 'a group', 'group','several dozen','scores','people','crowd','crowds']   # 50 -99
    medium_groups = ['busloads','hundreds','widespread','several hundred','large','hundred','protests','demonstrators','teachers','.']  #100 - 999
    large_groups = ['thousand','thousands','several thousand', 'mass', 'huge'] #1000-5000
    very_large_groups = ['tens of thousands']  #5000-10_000
    huge_groups =  ['government']        #10_000 - 100_000
    massive_groups = ['hundreds of thousands','millions','million','venezuela']    # >  100_000
    
    word_list = string.split(' ')
    
    word_list.reverse()   # to handle cases like hundreds of thousands
    
    for word in word_list:
        
        word = word.replace('\"','')
        
        if word in small_groups:
            return 50
        elif word in medium_groups:
            return 200
        elif word in large_groups:
            return 1500
        elif word in very_large_groups:
            return 20_000
        elif word in huge_groups:
            return 50_000
        elif word in massive_groups:
            return 600_000
    
    return string

In [777]:
def participants_to_categories(string):

    # if nan then just exit
    if string is np.nan:
        return string
    
    if type(string) is list:
        return string
    
    string = string.lower()
    
    small_groups = ['dozen','dozens', 'a group', 'group','several dozen','scores','people','crowd','crowds']   # 50 -99
    medium_groups = ['busloads','hundreds','widespread','several hundred','large','hundred','protests','demonstrators','teachers','.']  #100 - 999
    large_groups = ['thousand','thousands','several thousand', 'mass', 'huge'] #1000-5000
    very_large_groups = ['tens of thousands']  #5000-10_000
    huge_groups =  ['government']        #10_000 - 100_000
    massive_groups = ['hundreds of thousands','millions','million','venezuela']    # >  100_000
    
    word_list = string.split(' ')
    
    word_list.reverse()   # to handle cases like hundreds of thousands
    
    for word in word_list:
        
        word = word.replace('\"','')
        
        if word in small_groups:
            return '50-99'
        elif word in medium_groups:
            return '100-999'
        elif word in large_groups:
            return '1000-5000'
        elif word in very_large_groups:
            return '5000-10,000'
        elif word in huge_groups:
            return '10,000-100,000'
        elif word in massive_groups:
            return 'Over 100,000'
    
    return string

In [778]:

#mass['participants_converted'] = mass['participants']
mass['participants_converted'] = mass['participants'].apply(string_to_number)
mass['participants_converted'] = mass['participants_converted'].apply(participants_to_numbers)


mass['participants_cat_converted'] = mass['participants'].apply(string_to_number)
mass['participants_cat_converted'] = mass['participants_cat_converted'].apply(participants_to_categories)



In [779]:
mass['participants_category'].isnull().sum()

5352

In [780]:
#mass = mass[mass['participants_category'].notnull()]
mass = mass[mass['participants'].notnull()]

In [781]:
mass['participants'].isnull().sum()

0

In [782]:
mass.shape

(15227, 33)

In [783]:
mass['participants_category'].isnull().sum()

5345

In [784]:
mass['participants_converted'] = mass['participants_converted'].replace(',','', regex=True)

In [785]:
mass['participants_bad'] = [word if str(word).isnumeric() == np.nan or str(word).isnumeric() == False else '' for word in mass['participants_converted']]

In [786]:
print (f'There are {mass[mass["participants_bad"]!=""].shape[0] } bad rows in Mass dataframe.')

There are 0 bad rows in Mass dataframe.


In [787]:
mass[mass['participants_bad']!='']   [['country','year','protestnumber','participants_category','participants','participants_converted','protesterdemand1','stateresponse1']].iloc[0:50]

,country,year,protestnumber,participants_category,participants,participants_converted,protesterdemand1,stateresponse1


In [788]:
mass['participants_converted']= mass['participants_converted'].astype('int64') 

In [789]:
#mass.info()

In [790]:
mass[mass['participants_converted'] > 0]   [['country','year','participants_category','participants','participants_converted','participants_cat_converted','protesterdemand1','stateresponse1']].iloc[0:50]

,country,year,participants_category,participants,participants_converted,participants_cat_converted,protesterdemand1,stateresponse1
0,Canada,1990,NaN,1000s,1000,1000,"political behavior, process",ignore
1,Canada,1990,NaN,1000,1000,1000,"political behavior, process",ignore
2,Canada,1990,NaN,500,500,500,"political behavior, process",ignore
3,Canada,1990,NaN,100s,100,100,land farm issue,accomodation
4,Canada,1990,NaN,950,950,950,"political behavior, process",crowd dispersal
5,Canada,1990,NaN,200,200,200,police brutality,crowd dispersal
6,Canada,1991,NaN,110000,110000,110000,labor wage dispute,ignore
7,Canada,1991,NaN,110000,110000,110000,labor wage dispute,ignore
8,Canada,1992,NaN,1000,1000,1000,police brutality,arrests
9,Canada,1993,NaN,10000s,10000,10000,"political behavior, process",ignore


In [ ]:
mass.info